## import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pickle
import os

from IPython.display import display

from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


## Dataset Description

This dataset contains sales information from four stores of one of the retailers over 25 months.

Participants are expected to use these files to develop models that can predict customer demand. 

Additionally, the dataset includes a holdout sample with sales data for a 1-month period for which forecasts should be provided.

## sales.csv

Purpose: This file contains aggregated store sales for specific dates.

Columns:
date: Sales date

item_id: A unique identifier for each product

quantity: Total quantity of product sold per day

price_base: Average sales price per day

sum_total: Total daily sales amount

store_id: Store number

## online.csv

Purpose: This file contains aggregated online sales by store for specific dates.

Columns:

date: Sales date

item_id: A unique identifier for each product

quantity: Total quantity of product sold per day (online)

price_base: Average sales price per day

sum_total: Total daily sales amount

store_id: Store number

## markdowns.csv

Purpose: This file provides data on products sold at markdown prices in each store.

Columns:

date: Date of markdown

item_id: A unique identifier for each product

normal_price: Regular price

price: Price during markdown

quantity: Quantity sold at markdown

store_id: Store number

## price_history.csv

Purpose: This file contains price changes data in each store.

Columns:

date: Date of price change

item_id: A unique identifier for each product

price: Item new price

code: Price change code

store_id: Store number

## discounts_history.csv

Purpose: Contains historical promo data for each specific store.

Columns:

date: Date

item_id: A unique identifier for each product

sale_price_before_promo: Price before promo period started

sale_price_time_promo: Price during the promo period

promo_type_code: Promo code type

doc_id: Promo document number

number_disc_day: Sequential day number of the current promo period

store_id: Store number

## actual_matrix.csv

Purpose: Contains the list of products available in stores.

Columns:
item_id: A unique identifier for each product
date: Date of last product appearance in the current matrix
store_id: Store number

## catalog.csv

Purpose: Product catalog with characteristics.

Columns:

item_id: A unique identifier for each product

dept_name: Product department (hierarchy level)

class_name: Product class (hierarchy level)

subclass_name: Product subclass (hierarchy level)

item_type: Product type

weight_volume: Volumetric weight

weight_netto: Net weight

fatness: Fat content

## stores.csv

Purpose: Contains stores info data.

Columns:

store_id: Store number

division: Store division

format: Store format

city: Location

area: Store sales area

## test.csv

Purpose: Contains the test cases that participants are required to predict.

Columns:

row_id: Unique identifier for each record

item_id: A unique identifier for each product

store_id: Store number

date: The date for which the forecast should be made

## sample_submission.csv

Purpose: A sample submission file to demonstrate the expected format for the final submission.

Columns:

row_id: Unique identifier for each record

quantity: The predicted amount for the item. Initially set to 0.

## Load Data

In [2]:
df_sales = pd.read_csv("sales.csv", index_col='Unnamed: 0')
df_online = pd.read_csv('online.csv', index_col='Unnamed: 0')
df_markdowns = pd.read_csv('markdowns.csv', index_col='Unnamed: 0')
df_price_history = pd.read_csv('price_history.csv', index_col='Unnamed: 0')
df_discounts_history = pd.read_csv('discounts_history.csv', index_col='Unnamed: 0')
df_actual_matrix = pd.read_csv('actual_matrix.csv', index_col='Unnamed: 0')
df_catalog = pd.read_csv('catalog.csv', index_col='Unnamed: 0')
df_stores = pd.read_csv('stores.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('test.csv', sep = ';')
df_sample_submission = pd.read_csv('sample_submission.csv')

## Display first 2 rows of data

In [3]:
print('df_sales')
display(df_sales.head(2))

print('df_online')
display(df_online.head(2))

print('df_markdowns')
display(df_markdowns.head(2))

print('df_price_history')
display(df_price_history.head(2))

print('df_discounts_history')
display(df_discounts_history.head(2))

print('df_actual_matrix')
display(df_actual_matrix.head(2))

print('df_catalog')
display(df_catalog.head(2))

print('df_stores')
display(df_stores.head(2))

print('df_test')
display(df_test.head(2))

print('df_sample_submission')
display(df_sample_submission.head(2))

df_sales


,date,item_id,quantity,price_base,sum_total,store_id
0,2023-08-04,293375605257,1.0,47.86,47.86,1
1,2023-08-04,a66fdf2c0ae7,3.0,49.60,148.80,1


df_online


,date,item_id,quantity,price_base,sum_total,store_id
0,2023-08-04,4aa8dbe05246,3.0,12.40,37.21,1
1,2023-08-04,4e0fbcf99cf9,2.0,56.27,112.54,1


df_markdowns


...

,store_id,division,format,city,area
0,4,Div1,MaxiEuro,City3,1887
1,3,Div2,Format-7 express,City2,109


df_test


,row_id,item_id,store_id,date
0,0,c578da8e8841,1,27.09.2024
1,1,c578da8e8841,1,28.09.2024


df_sample_submission


,row_id,quantity
0,0,0
1,1,0


In [4]:
print('df_sales')
display(df_sales.shape)

print('df_online')
display(df_online.shape)

print('df_markdowns')
display(df_markdowns.shape)

print('df_price_history')
display(df_price_history.shape)

print('df_discounts_history')
display(df_discounts_history.shape)

print('df_actual_matrix')
display(df_actual_matrix.shape)

print('df_catalog')
display(df_catalog.shape)

print('df_stores')
display(df_stores.shape)

print('df_test')
display(df_test.shape)

print('df_sample_submission')
display(df_sample_submission.shape)

df_sales


(7432685, 6)

df_online


(1123412, 6)

df_markdowns


...

(4, 5)

df_test


(883680, 4)

df_sample_submission


(883680, 2)

In [5]:
df_sales.store_id.unique()

array([1, 2, 3, 4], dtype=int64)

## Step Preprocessing and Merging Data

In [6]:
# Merge sales data with catalog and store info

data = df_sales.merge(df_catalog, on="item_id", how="left").merge(df_stores, on="store_id", how="left")
data.head()

,date,item_id,quantity,price_base,sum_total,store_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness,division,format,city,area
0,2023-08-04,293375605257,1.000,47.86,47.86,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,0.0155,0.015,NaN,Div1,Format-1,City1,1500
1,2023-08-04,a66fdf2c0ae7,3.000,49.60,148.80,1,ЛИМОНАДЫ,НЕКОЛОСОДЕРЖАЩИЕ,"ДО 0,6Л.",Лимонады,0.5000,0.500,NaN,Div1,Format-1,City1,1500
2,2023-08-04,daa46ef49b7a,0.822,379.00,311.54,1,СЛОЙКА,СЛОЙКА,СОБСТВЕННОЕ ПРОИЗВОДСТВО,Слойки,NaN,NaN,NaN,Div1,Format-1,City1,1500
3,2023-08-04,a3b49c1bf758,1.000,129.00,129.00,1,ПЕРСОНАЛЬНЫЙ УХОД,ДЛЯ ВАННЫ И ДУША,ЖИДКОЕ МЫЛО,Жидкое Мыло,0.2000,0.200,NaN,Div1,Format-1,City1,1500
4,2023-08-04,ab611c5cef62,7.000,79.90,559.30,1,СОВРЕМЕННАЯ МОЛОЧНАЯ КАТЕГОРИЯ,ЙОГУРТЫ,ЙОГУРТЫ ЛОЖКОВЫЕ,Йогурты Ложковые,0.3500,0.350,1.5,Div1,Format-1,City1,1500


In [7]:
# Add online sales data

data = data.merge(df_online[["date", "item_id", "store_id", "quantity"]], 
                  on=["date", "item_id", "store_id"], 
                  how="left", 
                  suffixes=("", "_online"))
data.head()

,date,item_id,quantity,price_base,sum_total,store_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness,division,format,city,area,quantity_online
0,2023-08-04,293375605257,1.000,47.86,47.86,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,0.0155,0.015,NaN,Div1,Format-1,City1,1500,NaN
1,2023-08-04,a66fdf2c0ae7,3.000,49.60,148.80,1,ЛИМОНАДЫ,НЕКОЛОСОДЕРЖАЩИЕ,"ДО 0,6Л.",Лимонады,0.5000,0.500,NaN,Div1,Format-1,City1,1500,NaN
2,2023-08-04,daa46ef49b7a,0.822,379.00,311.54,1,СЛОЙКА,СЛОЙКА,СОБСТВЕННОЕ ПРОИЗВОДСТВО,Слойки,NaN,NaN,NaN,Div1,Format-1,City1,1500,NaN
3,2023-08-04,a3b49c1bf758,1.000,129.00,129.00,1,ПЕРСОНАЛЬНЫЙ УХОД,ДЛЯ ВАННЫ И ДУША,ЖИДКОЕ МЫЛО,Жидкое Мыло,0.2000,0.200,NaN,Div1,Format-1,City1,1500,NaN
4,2023-08-04,ab611c5cef62,7.000,79.90,559.30,1,СОВРЕМЕННАЯ МОЛОЧНАЯ КАТЕГОРИЯ,ЙОГУРТЫ,ЙОГУРТЫ ЛОЖКОВЫЕ,Йогурты Ложковые,0.3500,0.350,1.5,Div1,Format-1,City1,1500,NaN


In [8]:
# Add markdown data

data = data.merge(df_markdowns[["date", "item_id", "store_id", "quantity", "price"]], 
                  on=["date", "item_id", "store_id"], 
                  how="left", 
                  suffixes=("", "_markdown"))
data.head()

,date,item_id,quantity,price_base,sum_total,store_id,dept_name,class_name,subclass_name,item_type,weight_volume,weight_netto,fatness,division,format,city,area,quantity_online,quantity_markdown,price
0,2023-08-04,293375605257,1.000,47.86,47.86,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,0.0155,0.015,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN
1,2023-08-04,a66fdf2c0ae7,3.000,49.60,148.80,1,ЛИМОНАДЫ,НЕКОЛОСОДЕРЖАЩИЕ,"ДО 0,6Л.",Лимонады,0.5000,0.500,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN
2,2023-08-04,daa46ef49b7a,0.822,379.00,311.54,1,СЛОЙКА,СЛОЙКА,СОБСТВЕННОЕ ПРОИЗВОДСТВО,Слойки,NaN,NaN,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN
3,2023-08-04,a3b49c1bf758,1.000,129.00,129.00,1,ПЕРСОНАЛЬНЫЙ УХОД,ДЛЯ ВАННЫ И ДУША,ЖИДКОЕ МЫЛО,Жидкое Мыло,0.2000,0.200,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN
4,2023-08-04,ab611c5cef62,7.000,79.90,559.30,1,СОВРЕМЕННАЯ МОЛОЧНАЯ КАТЕГОРИЯ,ЙОГУРТЫ,ЙОГУРТЫ ЛОЖКОВЫЕ,Йогурты Ложковые,0.3500,0.350,1.5,Div1,Format-1,City1,1500,NaN,NaN,NaN


In [9]:
# Add price history
data = data.merge(df_price_history[["date", "item_id", "store_id", "price"]], 
                  on=["date", "item_id", "store_id"], 
                  how="left", 
                  suffixes=("", "_price_history"))

# Add discounts history
data = data.merge(df_discounts_history[["date", "item_id", "store_id", "promo_type_code"]], 
                  on=["date", "item_id", "store_id"], 
                  how="left")

# Add actual matrix data
data = data.merge(df_actual_matrix[["item_id", "store_id"]], on=["item_id", "store_id"], how="inner")

data.head()

,date,item_id,quantity,price_base,sum_total,store_id,dept_name,class_name,subclass_name,item_type,...,fatness,division,format,city,area,quantity_online,quantity_markdown,price,price_price_history,promo_type_code
0,2023-08-04,293375605257,1.0,47.86,47.86,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,...,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN,NaN,NaN
1,2023-08-07,293375605257,2.0,48.00,96.00,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,...,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN,NaN,NaN
2,2023-08-08,293375605257,4.0,48.00,192.00,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,...,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN,NaN,NaN
3,2023-08-17,293375605257,2.0,48.00,96.00,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,...,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN,NaN,NaN
4,2023-08-18,293375605257,1.0,48.00,48.00,1,ПРИКАССА,ЖЕВАТЕЛЬНАЯ РЕЗИНКА,ПОДУШЕЧКИ,Подушечки,...,NaN,Div1,Format-1,City1,1500,NaN,NaN,NaN,NaN,NaN


In [10]:
data.columns

Index(['date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id',
       'dept_name', 'class_name', 'subclass_name', 'item_type',
       'weight_volume', 'weight_netto', 'fatness', 'division', 'format',
       'city', 'area', 'quantity_online', 'quantity_markdown', 'price',
       'price_price_history', 'promo_type_code'],
      dtype='object')

## Select Training features

In [11]:
df_train = data[['date', 'item_id', 'store_id', 'quantity']]
df_train.head()

,date,item_id,store_id,quantity
0,2023-08-04,293375605257,1,1.0
1,2023-08-07,293375605257,1,2.0
2,2023-08-08,293375605257,1,4.0
3,2023-08-17,293375605257,1,2.0
4,2023-08-18,293375605257,1,1.0


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5864829 entries, 0 to 5864828
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   date      object 
 1   item_id   object 
 2   store_id  int64  
 3   quantity  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 179.0+ MB


In [13]:
df_train.isnull().sum()

date        0
item_id     0
store_id    0
quantity    0
dtype: int64

In [14]:
# Feature Engineering

# Convert date to datetime and extract features
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day
df_train['day_of_week'] = df_train['date'].dt.dayofweek

del df_train['date']

df_train.head()

,item_id,store_id,quantity,year,month,day,day_of_week
0,293375605257,1,1.0,2023,8,4,4
1,293375605257,1,2.0,2023,8,7,0
2,293375605257,1,4.0,2023,8,8,1
3,293375605257,1,2.0,2023,8,17,3
4,293375605257,1,1.0,2023,8,18,4


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5864829 entries, 0 to 5864828
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   item_id      object 
 1   store_id     int64  
 2   quantity     float64
 3   year         int32  
 4   month        int32  
 5   day          int32  
 6   day_of_week  int32  
dtypes: float64(1), int32(4), int64(1), object(1)
memory usage: 223.7+ MB


In [17]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to the column
df_train['item_id'] = label_encoder.fit_transform(df_train['item_id'])

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5864829 entries, 0 to 5864828
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   item_id      int32  
 1   store_id     int64  
 2   quantity     float64
 3   year         int32  
 4   month        int32  
 5   day          int32  
 6   day_of_week  int32  
dtypes: float64(1), int32(5), int64(1)
memory usage: 201.4 MB


In [24]:
# Prepare Data for Modeling
X = df_train.drop('quantity', axis = 1)
y = df_train['quantity']


In [25]:
# Train Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)


RandomForestRegressor(random_state=42)

In [26]:
df_test.head()

,row_id,item_id,store_id,date
0,0,c578da8e8841,1,27.09.2024
1,1,c578da8e8841,1,28.09.2024
2,2,c578da8e8841,1,29.09.2024
3,3,c578da8e8841,1,30.09.2024
4,4,c578da8e8841,1,01.10.2024


In [28]:
# Feature Engineering for test data

# Convert date to datetime and extract features
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day
df_test['day_of_week'] = df_test['date'].dt.dayofweek

# Apply LabelEncoder to the column
#df_test['item_id'] = label_encoder.transform(df_test['item_id'])

del df_test['date']

df_test.head()

,row_id,item_id,store_id,year,month,day,day_of_week
0,0,c578da8e8841,1,2024,9,27,4
1,1,c578da8e8841,1,2024,9,28,5
2,2,c578da8e8841,1,2024,9,29,6
3,3,c578da8e8841,1,2024,9,30,0
4,4,c578da8e8841,1,2024,10,1,1


In [29]:
# Ensure unseen labels in test data are handled

def safe_transform(encoder, data):
    # Get the known classes
    known_classes = set(encoder.classes_)
    # Replace unseen labels with a default value or NaN
    data = np.where(data.isin(known_classes), data, 'unknown')
    # Fit the LabelEncoder with the new "unknown" class if necessary
    if 'unknown' not in encoder.classes_:
        encoder.classes_ = np.append(encoder.classes_, 'unknown')
    return encoder.transform(data)

# Apply the LabelEncoder safely to the test data
df_test['item_id'] = safe_transform(label_encoder, df_test['item_id'])


In [30]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883680 entries, 0 to 883679
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   row_id       883680 non-null  int64
 1   item_id      883680 non-null  int32
 2   store_id     883680 non-null  int64
 3   year         883680 non-null  int32
 4   month        883680 non-null  int32
 5   day          883680 non-null  int32
 6   day_of_week  883680 non-null  int32
dtypes: int32(5), int64(2)
memory usage: 30.3 MB


In [32]:
df_test2 = df_test.copy(deep = True)

In [33]:
del df_test['row_id']

In [34]:
# Get prediction of quantity
y_pred = model.predict(df_test)

In [35]:
df_sample_submission['quantity'] = y_pred

df_sample_submission.to_csv('submit.csv', index=False)

df_sample_submission.head()

NameError: name 'submission' is not defined